In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/Resources/schools_complete.csv"
student_data_to_load = "Resources/Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
#Calculate the total number of schools
tot_schools = len(school_data)

#Calculate the total number of schools
tot_students = len(student_data)

#Calculate the total budget
tot_budget = school_data['budget'].sum()
#tot_budget

#Calculate the average math score
ave_math_score = student_data['math_score'].mean()
#round(ave_math_score,1)

#Calculate the average reading score
ave_reading_score = student_data['reading_score'].mean()
#round(ave_reading_score,1)

#Calculate the percentage of students with a passing math score (70 or greater)
tot_math_pass = len(student_data.loc[student_data['math_score']>=70])
per_math_pass = tot_math_pass/tot_students

#Calculate the percentage of students with a passing reading score (70 or greater)
tot_reading_pass = len(student_data.loc[student_data['reading_score']>=70])
per_reading_pass = tot_reading_pass/tot_students

## District Summary

Create a high level snapshot (in table form) of the district's key metrics, including:

* Total Schools
* Total Students
* Total Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)

In [3]:
df = pd.DataFrame({
    'Total Students':[tot_students],
    'Total Schools':[tot_schools],
    'Average Math Score':[ave_math_score],
    'Average Reading Score':[ave_reading_score], 
    'Percentage Passing Math':[100* per_math_pass],
    'Percentage Passing Reading':[100*per_reading_pass],
      'Average Passing Rate':[50*(per_math_pass+per_reading_pass)]
    
})
#df.reindex(labels=['Total District'])
df['Total Students'] = df['Total Students'].map("{:,}".format)
df['Average Math Score'] = df['Average Math Score'].map("{:.1f}".format)
df['Average Reading Score'] = df['Average Reading Score'].map("{:.1f}".format)
df['Average Passing Rate'] =  df['Average Passing Rate'].map("{:.1f}".format)
df['Percentage Passing Math'] =df['Percentage Passing Math'].map("{:.0f}%".format)
df['Percentage Passing Reading'] =df['Percentage Passing Reading'].map("{:.0f}%".format)
df

,Total Students,Total Schools,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Average Passing Rate
0,"39,170",15,79.0,81.9,75%,86%,80.4


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
student_mean = student_data.groupby(['school_name'])['reading_score','math_score'].mean()

sd_pass_math = school_data_complete.loc[ school_data_complete['math_score']>=70]

mp= sd_pass_math.groupby('school_name').count()[['math_score']]
sd_pass_read = school_data_complete.loc[ school_data_complete['reading_score']>=70]
rp= sd_pass_read.groupby('school_name').count()[['reading_score']]


In [5]:
school_data.head()
school_data_cons = school_data[['school_name','type','size','budget']]
school_data_cons['Per Student Funding']= school_data['budget']/school_data['size']
#school_data_cons['Average Math Score'] = student_data.groupby(['school_name']).mean()['math_score']
school_data_cons_m = pd.merge(school_data_cons,student_mean, how="left", on=["school_name", "school_name"])
school_data_cons_m  = school_data_cons_m.rename(columns={'math_score':'Average Math Score',
                                                         'reading_score':'Average Reading Score'
                                                         })
#school_data_cons_m['math_pass'] =mp['math_pass']
school_data_cons_m= school_data_cons_m.merge(mp,how='left',on=['school_name','school_name'])#,right_index= True)
school_data_cons_m= school_data_cons_m.merge(rp,how='left',on=['school_name','school_name'])#,right_index= True)
school_data_cons_m['Percentage Math Pass'] = school_data_cons_m['math_score']/school_data_cons_m['size']*100
school_data_cons_m['Percentage Reading Pass'] = school_data_cons_m['reading_score']/school_data_cons_m['size']*100
school_data_cons_m['Percentage Overall Pass'] = (school_data_cons_m['Percentage Math Pass'] + \
                                                school_data_cons_m['Percentage Reading Pass'])*0.5
school_data_cons_m =school_data_cons_m.drop(columns=['math_score','reading_score'])
sch_d_cons_unf = school_data_cons_m
school_data_cons_m['budget'] = school_data_cons_m['budget'].map("${:,}".format)
school_data_cons_m['Per Student Funding'] = school_data_cons_m['Per Student Funding'].map("${:.0f}".format)
school_data_cons_m['Average Reading Score'] =school_data_cons_m['Average Reading Score'].map("{:.1f}".format)
school_data_cons_m['Average Math Score'] =school_data_cons_m['Average Math Score'].map("{:.1f}".format)
school_data_cons_m['Percentage Reading Pass'] =school_data_cons_m['Percentage Reading Pass'].map("{:.1f}".format)
school_data_cons_m['Percentage Math Pass'] =school_data_cons_m['Percentage Math Pass'].map("{:.1f}".format)
school_data_cons_m['Percentage Overall Pass'] =school_data_cons_m['Percentage Overall Pass'].map("{:.1f}".format)
school_data_cons_m =school_data_cons_m.set_index('school_name')
school_data_cons_m


,type,size,budget,Per Student Funding,Average Reading Score,Average Math Score,Percentage Math Pass,Percentage Reading Pass,Percentage Overall Pass
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635",$655,81.2,76.6,65.7,81.3,73.5
Figueroa High School,District,2949,"$1,884,411",$639,81.2,76.7,66.0,80.7,73.4
Shelton High School,Charter,1761,"$1,056,600",$600,83.7,83.4,93.9,95.9,94.9
Hernandez High School,District,4635,"$3,022,020",$652,80.9,77.3,66.8,80.9,73.8
Griffin High School,Charter,1468,"$917,500",$625,83.8,83.4,93.4,97.1,95.3
Wilson High School,Charter,2283,"$1,319,574",$578,84.0,83.3,93.9,96.5,95.2
Cabrera High School,Charter,1858,"$1,081,356",$582,84.0,83.1,94.1,97.0,95.6
Bailey High School,District,4976,"$3,124,928",$628,81.0,77.0,66.7,81.9,74.3
Holden High School,Charter,427,"$248,087",$581,83.8,83.8,92.5,96.3,94.4


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
school_data_cons_m.sort_values('Percentage Overall Pass',ascending=False).head(5)

,type,size,budget,Per Student Funding,Average Reading Score,Average Math Score,Percentage Math Pass,Percentage Reading Pass,Percentage Overall Pass
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,84.0,83.1,94.1,97.0,95.6
Griffin High School,Charter,1468,"$917,500",$625,83.8,83.4,93.4,97.1,95.3
Pena High School,Charter,962,"$585,858",$609,84.0,83.8,94.6,95.9,95.3
Thomas High School,Charter,1635,"$1,043,130",$638,83.8,83.4,93.3,97.3,95.3
Wilson High School,Charter,2283,"$1,319,574",$578,84.0,83.3,93.9,96.5,95.2


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
school_data_cons_m.sort_values('Percentage Overall Pass',ascending=True).head(5)

,type,size,budget,Per Student Funding,Average Reading Score,Average Math Score,Percentage Math Pass,Percentage Reading Pass,Percentage Overall Pass
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,80.7,76.8,66.4,80.2,73.3
Figueroa High School,District,2949,"$1,884,411",$639,81.2,76.7,66.0,80.7,73.4
Huang High School,District,2917,"$1,910,635",$655,81.2,76.6,65.7,81.3,73.5
Johnson High School,District,4761,"$3,094,650",$650,81.0,77.1,66.1,81.2,73.6
Hernandez High School,District,4635,"$3,022,020",$652,80.9,77.3,66.8,80.9,73.8


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [8]:
school_data_complete['class']=pd.to_numeric(school_data_complete['grade'].str[:-2])
sch_by_grade =school_data_complete.groupby(['school_name','class'])

sch_by_grade['math_score'].mean().map("{:.1f}".format)


school_name            class
Bailey High School     9        77.1
                       10       77.0
                       11       77.5
                       12       76.5
Cabrera High School    9        83.1
                       10       83.2
                       11       82.8
                       12       83.3
Figueroa High School   9        76.4
                       10       76.5
                       11       76.9
                       12       77.2
Ford High School       9        77.4
                       10       77.7
                       11       76.9
                       12       76.2
Griffin High School    9        82.0
                       10       84.2
                       11       83.8
                       12       83.4
Hernandez High School  9        77.4
                       10       77.3
                       11       77.1
                       12       77.2
Holden High School     9        83.8
                       10       83.4
         

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
sch_by_grade['reading_score'].mean().map("{:.1f}".format)

school_name            class
Bailey High School     9        81.3
                       10       80.9
                       11       80.9
                       12       80.9
Cabrera High School    9        83.7
                       10       84.3
                       11       83.8
                       12       84.3
Figueroa High School   9        81.2
                       10       81.4
                       11       80.6
                       12       81.4
Ford High School       9        80.6
                       10       81.3
                       11       80.4
                       12       80.7
Griffin High School    9        83.4
                       10       83.7
                       11       84.3
                       12       84.0
Hernandez High School  9        80.9
                       10       80.7
                       11       81.4
                       12       80.9
Holden High School     9        83.7
                       10       83.3
         

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 625, 645, 800]
group_names = ["<$585", "$585-625", "$625-645", ">$645"]

In [11]:
sch_d_cons_unf['fund cut']=(pd.cut(pd.to_numeric(sch_d_cons_unf['Per Student Funding'].str[1:]),spending_bins,labels=group_names))
sch_d_cons_unf.dtypes

school_name                  object
type                         object
size                          int64
budget                       object
Per Student Funding          object
Average Reading Score        object
Average Math Score           object
Percentage Math Pass         object
Percentage Reading Pass      object
Percentage Overall Pass      object
fund cut                   category
dtype: object

In [15]:
sch_d_cons_unf[['Average Math Score','Average Reading Score','Percentage Math Pass','Percentage Reading Pass','Percentage Overall Pass']] = \
    sch_d_cons_unf[['Average Math Score','Average Reading Score','Percentage Math Pass','Percentage Reading Pass', 'Percentage Overall Pass']].apply(pd.to_numeric,errors='coerce',axis =1)
sch_d_cons_unf['Average Math Score'] = sch_d_cons_unf['Average Math Score'].map("{:.1f}".format)
sch_d_cons_unf['Average Reading Score'] = sch_d_cons_unf['Average Reading Score'].map("{:.1f}".format)
#sch_d_cons_unf['Average Passing Rate'] =  sch_d_cons_unf['Average Passing Rate'].map("{:.1f}".format)
#sch_d_cons_unf['Percentage Math Pass'] =sch_d_cons_unf['Percentage Math Pass'].map("{:.0f}%".format)
#sch_d_cons_unf['Percentage Reading Pass'] =sch_d_cons_unf['Percentage Reading Pass'].map("{:.0f}%".format)
sch_d_cons_unf.groupby('fund cut').mean()[['Average Math Score','Average Reading Score','Percentage Math Pass','Percentage Reading Pass','Percentage Overall Pass']]

KeyError: "['Average Math Score' 'Average Reading Score'] not in index"

## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2500, 3000, 5000]
group_names = ["Small Charter (<1000)", "Large Charter (1000-2500)", "Small District (2000-3500)", "Large Distric (>3500)"]
sch_d_cons_unf['size cut']=(pd.cut(sch_d_cons_unf['size'],size_bins,labels=group_names))
sch_d_cons_unf.groupby('size cut').mean()[['Average Math Score','Average Reading Score','Percentage Math Pass',
                'Percentage Reading Pass','Percentage Overall Pass']]

,Average Math Score,Average Reading Score,Percentage Math Pass,Percentage Reading Pass,Percentage Overall Pass
size cut,,,,,
Small Charter (<1000),83.800000,83.900000,93.550000,96.100000,94.850000
Large Charter (1000-2500),83.383333,83.883333,93.650000,96.733333,95.216667
Small District (2000-3500),76.800000,81.033333,66.666667,80.433333,73.566667
Large Distric (>3500),77.050000,80.900000,66.500000,81.050000,73.750000


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
sch_d_cons_unf.groupby('type').mean()[['Average Math Score','Average Reading Score','Percentage Math Pass',
                'Percentage Reading Pass','Percentage Overall Pass']]

,Average Math Score,Average Reading Score,Percentage Math Pass,Percentage Reading Pass,Percentage Overall Pass
type,,,,,
Charter,83.487500,83.887500,93.625000,96.575000,95.125000
District,76.942857,80.957143,66.571429,80.785714,73.671429


# Conclusions
* There are two dramatically different types of schools in this district: Large district schools and small charter schools. The largest charter school is around 2300 students and the smallest district school is around 2800 students. The charter schools outperform the district schools based on the specified criteria. 
* More analysis needs to be done on the reading scores. A difference of 2 points on an average score leads to a dramatically different result on the passing rate.
* There is a dramatic difference in the district schools performance in math and reading. While around 81% of students pass the reading section, 67% of students pass the math. 
* Scores seem are mostly stable across grades. They do not significantly change.


